## LangChain Expression Language(LCEL)

https://python.langchain.com/v0.1/docs/expression_language/why/

### 기본 구조: 프롬프트 + 모델 + 출력 파서


In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

In [ ]:
#API KEY 저장을 위한 os 라이브러리 호출
import os

os.environ['LANGCHAIN_PROJECT'] = 'LCEL'
print(f"[LANGCHAIN_PROJECT]\n{os.environ['LANGCHAIN_PROJECT']}")

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LCEL")

### 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿
  - `template`: 템플릿 문자열. 문자열 내에서 중괄호 `{}`는 변수를 나타냄
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의함

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트

In [10]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [ ]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

In [ ]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

In [ ]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

In [14]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini",
    max_tokens=2048,
    temperature=0.1,
)

### Chain 생성

#### LCEL(LangChain Expression Language)


```
chain = prompt | model | output_parser
```

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달


In [ ]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI()

chain = prompt | model
chain

### invoke() 호출

- python 딕셔너리 형태(키: 값)로 입력값을 전달
- invoke() 함수 호출 시, 입력값을 전달

In [18]:
# input 딕셔너리에 주제 설정
input = {"topic": "인공지능의 학습 방법"}

In [ ]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달
# 이를 통해 AI 모델이 생성한 메시지를 반환
chain.invoke(input)

In [ ]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

### 출력파서(Output Parser)


In [21]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [ ]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser
chain

In [ ]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능의 학습 원리"}
chain.invoke(input)

In [ ]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 가능
- `model_name` 역시 변경하여 테스트가 가능

In [26]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화
output_parser = StrOutputParser()

In [27]:
# 체인을 구성
chain = prompt | model | output_parser

In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

In [ ]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

### [실습] 템플릿을 변경하여 나만의 여행 가이드 챗봇 만들기

- 위의 프롬프트를 아래 주제에 맞게 **변경** 해보기
1. 페르소나: 10년차 여행 가이드
2. 3일간 가성비 여행 계획을 세워주는 챗봇 생성
3. `{question}` 에는 여행갈 나라와 도시를 사용자에게 입력받음
4. `answer` 변수를 출력하여 챗봇의 답변 결과 확인 
5. `Langsmith` 에 접속하여 실행 내용 확인

In [35]:
tour_guide_template = """

"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성


# ChatOpenAI 챗모델을 초기화


# 문자열 출력 파서를 초기화


# 체인을 구성


In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청

# 스트리밍 출력


In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청

# 스트리밍 출력


### [실습] 템플릿을 변경하여 나만의 요리사 챗봇 만들기

- 위의 프롬프트를 아래 주제에 맞게 **변경** 해보기
1. 페르소나: 10년차 셰프
2. 냉장고 속 재료(여러 재료도 가능)를 입력으로 받아 요리명과 레시피 출력
3. `{food}` 에는 여행갈 냉장고 속 재료를 사용자에게 입력받음
4. `answer` 변수를 출력하여 챗봇의 답변 결과 확인 
5. `Langsmith` 에 접속하여 실행 내용 확인

In [ ]:
# 코드 작성

### [실습] 템플릿을 변경하여 나만의 헬스 트레이너 챗봇 만들기

- 위의 프롬프트를 아래 주제에 맞게 **변경** 해보기
1. 페르소나: 10년차 헬스 트레이너
2. 운동하고 싶은 신체 부위를 입력하면 운동 루틴을 출력 
3. `{today}` 에는 운동하고 싶은 신체 부위를 사용자에게 입력받음
4. `answer` 변수를 출력하여 챗봇의 답변 결과 확인 
5. `Langsmith` 에 접속하여 실행 내용 확인

In [ ]:
# 코드 작성